In [1]:
!pip install fuzzywuzzy

In [2]:
!pip install git+https://github.com/openai/openai-python

  Cloning https://github.com/openai/openai-python to /tmp/pip-req-build-7bunsid5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/openai-python /tmp/pip-req-build-7bunsid5
  Resolved https://github.com/openai/openai-python to commit 93aa548201af70646fe651cc01fd875f32b00202
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
  Created wheel for openai: filename=openai-1.44.0-py3-none-any.whl size=367790 sha256=99a12bbda15f9c632ee911851e101669a82ee34992fbfd658a9db27d90dc0a9c
  Stored in directory: /tmp/pip-ephem-wheel-cache-aqy69765/wheels/eb/79/d9/d890c3e8e42635d76

In [3]:
import re
import pandas as pd
import json
import os
import ast
from fuzzywuzzy import fuzz
from openai import OpenAI

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
import os
os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

In [5]:
print(os.environ.get('OPENAI_API_KEY'))

YOUR_API_KEY


In [6]:
client = OpenAI(
    api_key="YOUR_API_KEY",
)

## Preprocessing dataset

In [ ]:
# Function to extract text between tags and remove subtags
def extract_text(tag, text):
    pattern = rf'<{tag}>(.*?)</{tag}>'
    match = re.search(pattern, text)
    if match:
        # Remove subtags
        return re.sub(r'<.*?>', '', match.group(1)).strip()
    else:
        return None

# Read citation strings from text file
with open('/content/500_citation.txt', 'r') as file:
    citation_strings = file.read()

# Extract metadata fields
metadata = ['author', 'title', 'issued', 'container-title', 'editor', 'publisher']
data = []
for citation_string in citation_strings.split('\n'):
    fields = {}
    for field in metadata:
        fields[field] = extract_text(field, citation_string)
    fields = {k: v if pd.notna(v) else '' for k, v in fields.items()}
    fields['citation_string'] = ', '.join(fields.values())

    data.append(fields)

# Create DataFrame
df = pd.DataFrame(data)
df

,author,title,issued,container-title,editor,publisher,citation_string
0,,New Names in Volume 52,1978,Journal of Helminthology,,,", New Names in Volume 52, 1978, Journal of Hel..."
1,"Cheng M, Chen ZB, Xing JW","Design, Analysis, and Experimental Evaluation ...",2018,IEEE Trans Magn,,,"Cheng M, Chen ZB, Xing JW, Design, Analysis, a..."
2,Schütz T.,Der aerodynamische Entwicklungsprozess,2016,Fahrzeugaerodynamik,,Springer Fachmedien Wiesbaden,"Schütz T., Der aerodynamische Entwicklungsproz..."
3,"Matthew W. Veal., Scott A. Shearer., John P. F...",Improved Mass Flow Sensing for Yield Monitorin...,2004,"2004, Ottawa, Canada August 1 - 4, 2004",,American Society of Agricultural and Biologica...,"Matthew W. Veal., Scott A. Shearer., John P. F..."
4,C. Tengelmann,,1964,Das Recht Des Einkaufs,,Gabler Verlag,"C. Tengelmann, , 1964, Das Recht Des Einkaufs,..."
...,...,...,...,...,...,...,...
501,,TVOs,2018,Encyclopedia of Ophthalmology,,Springer Berlin Heidelberg,", TVOs, 2018, Encyclopedia of Ophthalmology, ,..."
502,,,1868,Sci Am,,,", , 1868, Sci Am, ,"
503,Helm F. and Klein E.,Malignant: Cutaneous clues to malignant disease,1976,Plastic and Reconstructive Surgery,,,"Helm F. and Klein E., Malignant: Cutaneous clu..."
504,"Godbole, R.V., Rao, Pratibha, Alegaonkar, P.S....",“Influence of fuel to oxidizer ratio on LPG se...,July 2015,Materials Chemistry and Physics,,Elsevier BV,"Godbole, R.V., Rao, Pratibha, Alegaonkar, P.S...."


## Refromatting 1K of GIANT1B dataset for fine tuning GPT-4o-mini & splitting dataset to train and validation

In [ ]:
system_message = (
    "You are an advanced Citation Parsing system designed for extracting metadata from citation strings. "
    "Your task is to identify and extract specific metadata fields from various citation styles, including IEEE, ACM, APA, MLA, Chicago, and so on. "
    "The metadata fields are defined as follows: "
    "a) Title: The title of the paper or article. "
    "b) Author: The names of the authors involved in the publication. "
    "c) Container-Title: The name of the publication venue, such as a journal name, conference proceedings, book title, thesis, or technical report. "
    "For example, if the cited work is an article, the 'container-title' would be the journal name or the name of the conference proceedings where the article was published. "
    "d) Issued: The date of publication, including year, month, and day if available. Do not get confused with the volume number. Ignore the volume number and page number. "
    "e) Editor: The names of the editors who contributed to a book, journal, or other publication. These often include terms like 'eds.', 'editors', '(eds.)', or '(editors)' after the names. "
    "f) Publisher: The organization or company responsible for producing and distributing the publication. "
    "Your goal is to accurately extract these fields from the provided citation strings."
)

def create_user_message(row):
    # Creates the user message with the citation string
    return f"{row['citation_string']}"

def create_assistant_message(row):
    # Creates the assistant's response with metadata fields
    return (
        f"author: '{row['author']}'\n"
        f"title: '{row['title']}'\n"
        f"issued: '{row['issued']}'\n"
        f"container-title: '{row['container-title']}'\n"
        f"editor: '{row['editor']}'\n"
        f"publisher: '{row['publisher']}'"
    )

def prepare_example_message(row):
    # Prepares the final message format in JSON
    return {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": create_user_message(row)},
            {"role": "assistant", "content": create_assistant_message(row)}
        ]
    }


In [ ]:
# training_data = df.apply(prepare_example_message, axis=1).tolist()
# training_data[1:10]
train = df.loc[0:200]
training_data = train.apply(prepare_example_message, axis=1).tolist()
training_data[1:10]

[{'messages': [{'role': 'system',
    'content': "You are an advanced Citation Parsing system designed for extracting metadata from citation strings. Your task is to identify and extract specific metadata fields from various citation styles, including IEEE, ACM, APA, MLA, Chicago, and so on. The metadata fields are defined as follows: a) Title: The title of the paper or article. b) Author: The names of the authors involved in the publication. c) Container-Title: The name of the publication venue, such as a journal name, conference proceedings, book title, thesis, or technical report. For example, if the cited work is an article, the 'container-title' would be the journal name or the name of the conference proceedings where the article was published. d) Issued: The date of publication, including year, month, and day if available. Do not get confused with the volume number. Ignore the volume number and page number. e) Editor: The names of the editors who contributed to a book, journal, o

### Counting total number of tokens in a training data

In [ ]:
# Function to count tokens by splitting on spaces
def count_tokens_basic(text):
    return len(text.split())

# Function to sum total tokens in all examples
def count_total_tokens(training_data):
    total_tokens_all = 0

    for idx, example in enumerate(training_data):
        total_tokens_example = 0

        # Iterate through each message in the 'messages' list
        for message in example['messages']:
            content = message['content']

            # Count tokens in the 'content' field
            num_tokens = count_tokens_basic(content)
            total_tokens_example += num_tokens

        print(f"Total tokens in example {idx + 1}: {total_tokens_example}")
        total_tokens_all += total_tokens_example

    print(f"\nTotal number of tokens across all examples: {total_tokens_all}")

# Call the function on your training data
count_total_tokens(training_data)

Total tokens in example 1: 225
Total tokens in example 2: 251
Total tokens in example 3: 224
Total tokens in example 4: 274
Total tokens in example 5: 224
Total tokens in example 6: 229
Total tokens in example 7: 275
Total tokens in example 8: 237
Total tokens in example 9: 265
Total tokens in example 10: 274
Total tokens in example 11: 221
Total tokens in example 12: 300
Total tokens in example 13: 300
Total tokens in example 14: 225
Total tokens in example 15: 219
Total tokens in example 16: 232
Total tokens in example 17: 260
Total tokens in example 18: 301
Total tokens in example 19: 233
Total tokens in example 20: 249
Total tokens in example 21: 261
Total tokens in example 22: 256
Total tokens in example 23: 260
Total tokens in example 24: 243
Total tokens in example 25: 228
Total tokens in example 26: 245
Total tokens in example 27: 235
Total tokens in example 28: 237
Total tokens in example 29: 239
Total tokens in example 30: 235
Total tokens in example 31: 223
Total tokens in e

In [ ]:
validation_df = df.loc[201:221]
validation_data = validation_df.apply(prepare_example_message, axis=1).tolist()
validation_data[1:10]

[{'messages': [{'role': 'system',
    'content': "You are an advanced Citation Parsing system designed for extracting metadata from citation strings. Your task is to identify and extract specific metadata fields from various citation styles, including IEEE, ACM, APA, MLA, Chicago, and so on. The metadata fields are defined as follows: a) Title: The title of the paper or article. b) Author: The names of the authors involved in the publication. c) Container-Title: The name of the publication venue, such as a journal name, conference proceedings, book title, thesis, or technical report. For example, if the cited work is an article, the 'container-title' would be the journal name or the name of the conference proceedings where the article was published. d) Issued: The date of publication, including year, month, and day if available. Do not get confused with the volume number. Ignore the volume number and page number. e) Editor: The names of the editors who contributed to a book, journal, o

### Count the number of tokens in the validation file

In [ ]:
# Function to count tokens by splitting on spaces
def count_tokens_basic(text):
    return len(text.split())

# Function to sum total tokens in all examples
def count_total_tokens(validation_data):
    total_tokens_all = 0

    for idx, example in enumerate(validation_data):
        total_tokens_example = 0

        # Iterate through each message in the 'messages' list
        for message in example['messages']:
            content = message['content']

            # Count tokens in the 'content' field
            num_tokens = count_tokens_basic(content)
            total_tokens_example += num_tokens

        print(f"Total tokens in example {idx + 1}: {total_tokens_example}")
        total_tokens_all += total_tokens_example

    print(f"\nTotal number of tokens across all examples: {total_tokens_all}")

# Call the function on your training data
count_total_tokens(validation_data)

Total tokens in example 1: 235
Total tokens in example 2: 247
Total tokens in example 3: 235
Total tokens in example 4: 272
Total tokens in example 5: 278
Total tokens in example 6: 225
Total tokens in example 7: 228
Total tokens in example 8: 227
Total tokens in example 9: 227
Total tokens in example 10: 237
Total tokens in example 11: 213
Total tokens in example 12: 230
Total tokens in example 13: 223
Total tokens in example 14: 255
Total tokens in example 15: 228
Total tokens in example 16: 239
Total tokens in example 17: 232
Total tokens in example 18: 221
Total tokens in example 19: 265
Total tokens in example 20: 233
Total tokens in example 21: 279

Total number of tokens across all examples: 5029


### We need to upload the example as Json object, where each line being one training example

In [ ]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [ ]:
training_file_name = "citation_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "citation_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [ ]:
!head -n 5 citation_finetune_training.jsonl

{"messages": [{"role": "system", "content": "You are an advanced Citation Parsing system designed for extracting metadata from citation strings. Your task is to identify and extract specific metadata fields from various citation styles, including IEEE, ACM, APA, MLA, Chicago, and so on. The metadata fields are defined as follows: a) Title: The title of the paper or article. b) Author: The names of the authors involved in the publication. c) Container-Title: The name of the publication venue, such as a journal name, conference proceedings, book title, thesis, or technical report. For example, if the cited work is an article, the 'container-title' would be the journal name or the name of the conference proceedings where the article was published. d) Issued: The date of publication, including year, month, and day if available. Do not get confused with the volume number. Ignore the volume number and page number. e) Editor: The names of the editors who contributed to a book, journal, or oth

In [ ]:
!head -n 5 citation_finetune_validation.jsonl

{"messages": [{"role": "system", "content": "You are an advanced Citation Parsing system designed for extracting metadata from citation strings. Your task is to identify and extract specific metadata fields from various citation styles, including IEEE, ACM, APA, MLA, Chicago, and so on. The metadata fields are defined as follows: a) Title: The title of the paper or article. b) Author: The names of the authors involved in the publication. c) Container-Title: The name of the publication venue, such as a journal name, conference proceedings, book title, thesis, or technical report. For example, if the cited work is an article, the 'container-title' would be the journal name or the name of the conference proceedings where the article was published. d) Issued: The date of publication, including year, month, and day if available. Do not get confused with the volume number. Ignore the volume number and page number. e) Editor: The names of the editors who contributed to a book, journal, or oth

### Upload the training and validation files for fine-tuning

In [ ]:
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id


training_file_id = upload_file(training_file_name, "fine-tune")
validation_file_id = upload_file(validation_file_name, "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-AQg3yRh3d20w5WKEs3GOlBu5
Validation file ID: file-8bJghrsIbnGskj8V0hTjfAae


### Fine-tuning using openAI API

In [ ]:
MODEL = "gpt-4o-mini-2024-07-18"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=MODEL,
    suffix="lmparscit-gpt4omini",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

BadRequestError: Error code: 400 - {'error': {'message': 'Fine-tuning jobs cannot be created on an Explore plan. You can upgrade to a paid plan on your billing page: https://platform.openai.com/account/billing/overview', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}

Because of the quota limit in the explore plan, we could not fine-tune

## Processing Evaluation Dataset -- Cora Ref
Cora Ref consists of 500 computer science conference and journals citation strings.

In [ ]:
def extract_text_from_tags(citation_string):
    # Define regular expression patterns for each tag
    tag_patterns = {
        'author': r'<author>(.*?)<\/author>',
        'title': r'<title>(.*?)<\/title>',
        'editor': r'<editor>(.*?)<\/editor>',
        'venue': r'<(?:journal|booktitle)>(.*?)<\/(?:journal|booktitle)>',
        'volume': r'<volume>(.*?)<\/volume>',
        'date': r'<date>(.*?)<\/date>'
    }

    # Initialize dictionary to store extracted text
    extracted_text = {}

    # Extract text for each tag
    for tag, pattern in tag_patterns.items():
        matches = re.findall(pattern, citation_string)
        if matches:
            extracted_text[tag] = matches[0]
        else:
            extracted_text[tag] = None

    return extracted_text

def main():
    # Read CORA Ref citation strings from a text file
    with open('/content/tagged_references.txt', 'r') as file:
        citation_strings = file.read().splitlines()

    # Initialize list to store extracted data
    data = []

    # Extract text from each citation string
    for citation_string in citation_strings:
        extracted_data = extract_text_from_tags(citation_string)

        # Extract citation string without tags
        citation_only = re.sub(r'<\/?[\w]+>', '', citation_string)
        extracted_data['CitationString'] = citation_only.strip()

        data.append(extracted_data)

    # Create DataFrame from the extracted data
    df = pd.DataFrame(data)

    # Merge 'journal' and 'booktitle' into 'Venue' column
    if 'journal' in df.columns and 'booktitle' in df.columns:
        df['venue'] = df[['journal', 'booktitle', ]].fillna('').sum(axis=1)
        df.drop(columns=['journal', 'booktitle'], inplace=True)

    # Save DataFrame to CSV
    df.to_csv('CORA-for-eval.csv', index=False)

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import re

# Read citation strings from a text file
with open("/content/tagged_references.txt", "r") as file:
    citation_strings = file.readlines()

# Function to extract text between tags
def extract_text(tag, string):
    pattern = fr"<{tag}>(.*?)</{tag}>"
    matches = re.findall(pattern, string, re.DOTALL)
    return matches[0].strip() if matches else ""

# Extract text between tags for each citation string
citation_data = []
for string in citation_strings:
    author = extract_text("author", string)
    title = extract_text("title", string)
    editor = extract_text("editor", string)
    booktitle = extract_text("booktitle", string)
    journal = extract_text("journal", string)
    volume = extract_text("volume", string)
    date = extract_text("date", string)
    publisher = extract_text("publisher", string)
    tech = extract_text("tech", string)
    institution = extract_text("institution", string)
    location = extract_text("location", string)

    # Combine journal, booktitle, publisher, and institution into a single venue column
    venue = journal or booktitle or publisher or institution

    citation_data.append({
        "CitationString": string.strip(),
        "author": author,
        "title": title,
        "editor": editor,
        "venue": venue,
        "volume": volume,
        "date": date,
        "tech": tech,
        "location": location
    })

# Create dataframe from the extracted data
citation_df = pd.DataFrame(citation_data)

# Save dataframe to CSV file
citation_df.to_csv("citation_venue_mapped.csv", index=False)

print("Dataframe created and saved successfully!")



Dataframe created and saved successfully!


In [ ]:
dataframe = pd.read_csv('CORA-for-eval.csv')
dataframe.head()

,author,title,editor,venue,volume,date,CitationString
0,"A. Cau, R. Kuiper, and W.-P. de Roever.",Formalising Dijkstra's development strategy w...,"In C. B. Jones, R. C. Shaw, and T. Denvir, ed...","Proc. 5th. BCS-FACS Refinement Workshop,",NaN,1992.,"A. Cau, R. Kuiper, and W.-P. de Roever. Form..."
1,"M. Kitsuregawa, H. Tanaka, and T. Moto-oka.",Application of hash to data base machine and ...,NaN,"New Generation Computing,","1(1),",1983.,"M. Kitsuregawa, H. Tanaka, and T. Moto-oka. ..."
2,Alexander Vrchoticky.,Modula/R language definition.,NaN,NaN,NaN,May 1993.,Alexander Vrchoticky. Modula/R language defi...
3,Marc Shapiro and Susan Horwitz.,Fast and accurate flow-insensitive points-to ...,NaN,In Proceedings of the 24th Annual ACM Symposi...,NaN,January 1997.,Marc Shapiro and Susan Horwitz. Fast and acc...
4,W. Landi and B. G. Ryder.,Aliasing with and without pointers: A problem...,NaN,NaN,NaN,September 1990.,W. Landi and B. G. Ryder. Aliasing with and ...


## Prompt Engineering using Few-Shot Learning:

We need:
a) context (i.e., the particular task (e.g., Citation Parsing AI assistant) we want to achieve)
b) examples (e.g., provide variety of citation string examples)
c) task (e.g. provide any test examples)
d) output

In [ ]:
SYS_PROMPT = """You are a smart and intelligent Citation Parsing system (i.e., very similar to Named Entity Recognition (NER) tasks or information extraction or metadata extraction or knowledge extraction).
I will provide you the definition of the metadata fields from which you need to extract the metadata fields from a given citation string in different styles (e.g., IEEE, ACM, APA, MLA, Chicago, etc.).
I will also provide you with the output in given format with examples."""

USER_PROMPT_1 = "Are you clear about your role?"

ASSIST_PROMPT_1 = "Sure, I am ready to help you with your citation parsing task where the citation strings may have adopted different citation styles (e.g., IEEE, ACM, APA, MLA, Chicago, etc.)"

PROMPT = (
    "Metadata Fields Definition\n"
    "1. title: The title of the paper/article.\n"
    "2. author: List of authors involved in the publication.\n"
    "3. venue: Information about the publication venue. Depending on the source, this could be a journal name, conference proceedings, book title, thesis, technical report, etc.\n"
    "4. date: The year, month, and day of publication.\n"
    "\n"
    "Output Format:\n"
    "{'title': [list of metadata fields present], 'author': [list of metadata fields present], 'venue': [list of metadata fields present], 'date': [list of metadata fields present]}\n"
    "If no metadata fields are presented in any categories, keep it None\n"
    "\n"
    "Examples:\n"
    "\n"
    "1. Citation String: K. A. Bowman and J. D. Meindl. 2001. Impact of within-die parameter fluctuations on future maximum clock frequency distributions, Proceedings of the IEEE Custom Integrated  Circuits Conference, 229-232.\n"
    "Output: {'title': ['Impact of within-die parameter fluctuations on future maximum clock frequency distributions'], 'author': ['K. A. Bowman', 'J. D. Meindl'], 'venue': ['Proceedings of the IEEE Custom Integrated  Circuits Conference'], 'date': ['2001']}\n"
    "\n"
    "2. Citation String: Elghadamsi, F. E., and Mohraz, B., “Inelastic Earthquake Spectra,” Earthquake Engineering and Structural Dynamics. Vol. 15, 1987.\n"
    "Output: {'title': ['Inelastic Earthquake Spectra'], 'author': ['Elghadamsi, F. E.', 'Mohraz, B.'], 'venue': ['Earthquake Engineering and Structural Dynamics'], 'date': ['1987']}\n"
    "\n"
    "3. Citation String: Mataric. M. J. (1990). A distributed model for mobile robot environment-learning and navigation. Master's thesis. MIT. Cambridge. MA.\n"
    "Output: {'title': ['A distributed model for mobile robot environment-learning and navigation'], 'author': ['Mataric. M. J.'], 'venue': ['MIT'], 'date': ['1990']}\n"
    "\n"
    "4. Citation String: Mettala. E. and Graham. M. (1992). The domain-specific software architecture program. Technical Report CMU/SEI-92-TR-22 ESD-92-TR-223. Carnegie Mellon Software Engineering Institute.\n"
    "Output: {'title': ['The domain-specific software architecture program'], 'author': ['Mettala. E.', 'Graham. M.'], 'venue': ['Carnegie Mellon Software Engineering Institute'], 'date': ['1992']}\n"
    "\n"
    "5. Citation String: W. Landi and B. G. Ryder. Aliasing with and without pointers: A problem taxonomy. Center for Computer Aids for Industrial Productivity Technical Report CAIP-TR-125, Rutgers University, September 1990.\n"
    "Output: {'title': ['Aliasing with and without pointers: A problem taxonomy'], 'author': ['W. Landi and B. G. Ryder'], 'venue': ['Rutgers University'], 'date': ['September 1990']}\n"
    "6. Citation String: {}\n"
    "Output: {'title': None, 'author': None, 'venue': None, 'date': None}"
)


In [ ]:
MODEL = "gpt-4o-mini"
def openai_chat_completion_response(final_prompt):
  completion = openai.chat.completions.create(
      model = MODEL,
      messages = [
          {"role": "system", "content": SYS_PROMPT},
          {"role": "user", "content": final_prompt},
          # {"role": "assistant", "content": ASSIST_PROMPT_1},
          # {"role": "user", "content": final_prompt}

      ]
  )

  #return response['choices'][0]['message']['content'].strip("\n")
  return completion.choices[0].message.content.strip("\n")

In [ ]:
citedata = pd.read_csv('/content/CORA-for-eval.csv', encoding='utf-8')
result = []

# Iterate through each citation string in the DataFrame
for citation in citedata['CitationString']:
    # Replace the placeholder in the PROMPT with the actual citation string
    prompt = PROMPT.replace('{}', citation)

    # Perform the OpenAI completion using the modified prompt
    parsing = openai_chat_completion_response(prompt)

    # Append the result to the list
    result.append(parsing)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
result

["Got it. Please provide a citation string, and I'll extract the metadata fields based on the given definitions and format.",
 'Got it. Please provide the citation string you would like me to parse.',
 'Got it. Please provide the citation string that you would like me to parse.',
 'Got it! Please provide the citation string you would like me to parse according to the defined metadata fields.',
 'Understood. Please provide the citation string you would like me to parse, and I will extract the metadata accordingly.',
 "Got it. Please provide me with the citation strings and I'll parse them to extract the required metadata fields according to the specified definitions and format.",
 "Got it. Please provide a citation string, and I will parse it to extract the metadata fields according to the definitions and formats you've given.",
 "Got it. Let's proceed with parsing the citations you provide according to the defined metadata fields and output format. Please provide a citation string you 

In [ ]:
dataframe = pd.DataFrame(result, columns = ['metadata'])
dataframe.to_csv("predicted-gpt4o-cora.csv", index = False)

### Parsing Predicted Data (JSON output of GPT model)

In [ ]:
import pandas as pd
import json

# Load the CSV file
df = pd.read_csv('/content/predicted-gpt-cora.csv')

# Function to fix JSON data
def fix_json(json_str):
    try:
        # Replace single quotes with double quotes
        json_str = json_str.replace("'", '"')
        # Handle apostrophes in the JSON string
        json_str = json_str.replace('"s', "'s")
        # Handle single quotes in the venue field
        json_str = json_str.replace("'90", "'90")
        json_str = json_str.replace("'97", "'97")
        json_str = json_str.replace("'s", "'s")
        json_str = json_str.replace('(ASPLOS-VI)"', "(ASPLOS-VI)'")
        # Handle None values
        json_str = json_str.replace("[None]", '["None"]')
        json_str = json_str.replace("None", '["None"]')
        # Load JSON string
        return json.loads(json_str)
    except (json.JSONDecodeError, TypeError):
        return {}

# Fix JSON data under the 'metadata' column
df['metadata'] = df['metadata'].apply(fix_json)

# Normalize JSON data and save in a new DataFrame
normalized_data = pd.json_normalize(df['metadata']).fillna('')

# Save the result to a new CSV file
normalized_data.to_csv('predicted_result.csv', index=False)

###Evaluation

In [ ]:
eval_data = pd.read_csv('/content/predicted_result-updated.csv')
eval_data

,title,author,venue,date
0,"[""Formalising Dijkstra's development strategy ...","['A. Cau', 'R. Kuiper', 'W.-P. de Roever']",['Proc. 5th. BCS-FACS Refinement Workshop'],['1992']
1,['Application of hash to data base machine and...,"['M. Kitsuregawa', 'H. Tanaka', 'T. Moto-oka']",['New Generation Computing'],['1983']
2,['Modula/R language definition'],['Alexander Vrchoticky'],"['Technical Report TU Wien rr-02-92, version 2...",['May 1993']
3,['Fast and accurate flow-insensitive points-to...,"['Marc Shapiro', 'Susan Horwitz']",['Proceedings of the 24th Annual ACM Symposium...,['January 1997']
4,['Aliasing with and without pointers: A proble...,"['W. Landi', 'B. G. Ryder']",['Center for Computer Aids for Industrial Prod...,['September 1990']
...,...,...,...,...
495,['Semantic parallels in natural language and c...,"['van Benthem, J. F. A. K.']","['Logic Colloquium'87, Granada']",['1989']
496,['Bias-driven revision of logical domain theor...,"['Koppel, M.', 'Feldman, R.', 'Segre, A. M.']",['Journal of Artificial Intelligence Research'],['1994b']
497,['Learning decision lists by prepending inferr...,"['Webb, G. I.', 'Brkic, N.']",['Proceedings of the Australian Workshop on Ma...,['1993']
498,['Integrating Reflection into SLD-Resolution'],"['Barklund, J.', 'Costantini, S.', ""Dell'Acqua...",['Proc. Post-Conf. Ws. on Proof-Theoretical Ex...,['1994']


## Title Field Evaluation

In [ ]:
pred_title = []
for string in eval_data['title']:
  strings = string.strip()
  strings = strings.strip("['").strip("']").strip('"')
  strings = strings.lower()
  pred_title.append(strings)

pred_data_title = pd.DataFrame(pred_title, columns = ['pred-title'])

In [ ]:
pred_data_title

,pred-title
0,formalising dijkstra's development strategy wi...
1,application of hash to data base machine and i...
2,modula/r language definition
3,fast and accurate flow-insensitive points-to a...
4,aliasing with and without pointers: a problem ...
...,...
495,semantic parallels in natural language and com...
496,bias-driven revision of logical domain theories
497,learning decision lists by prepending inferred...
498,integrating reflection into sld-resolution


In [ ]:
test_data = pd.read_csv('CORA-for-eval.csv')
test_data['title'].fillna('None', inplace=True)

In [ ]:
test_title = []
characters_to_remove = ['.', ',', '"', "''", '``']
for string in test_data['title']:
  cleaned_string = string.strip().strip(''.join(characters_to_remove))
  normalize = cleaned_string.lower()
  test_title.append(normalize)

test_data_title = pd.DataFrame(test_title, columns = ['test-title'])

In [ ]:
test_data_title

,test-title
0,formalising dijkstra's development strategy wi...
1,application of hash to data base machine and i...
2,modula/r language definition
3,fast and accurate flow-insensitive points-to a...
4,aliasing with and without pointers: a problem ...
...,...
495,semantic parallels in natural language and com...
496,bias-driven revision of logical domain theories
497,learning decision lists by prepending inferred...
498,integrating reflection into sld-resolution


In [ ]:
matched = pd.DataFrame(columns=['title_match'])
matched['title_match'] = pred_data_title['pred-title'].eq(test_data_title['test-title']).replace([True, False], [1,0])
result = pd.concat([pred_data_title, test_data_title, matched], axis = 1, sort = False)
result.to_csv("title_output.csv", index=False)

In [ ]:
print(matched.value_counts())

title_match
1              480
0               20
Name: count, dtype: int64


## Fuzzy Matching For Title Field

In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd


# Function to perform fuzzy string matching
def fuzzy_match(str1, str2):
    return fuzz.token_sort_ratio(str1.lower(), str2.lower())

# Perform string matching and fuzzy matching
matched = []
for pred_title in pred_data_title['pred-title']:
    match_found = False
    for test_title in test_data_title['test-title']:
        if pred_title.lower() == test_title.lower():  # Exact match
            matched.append(1)
            match_found = True
            break
        elif fuzzy_match(pred_title, test_title) >= 90:  # Fuzzy match with threshold 90%
            matched.append(1)
            match_found = True
            break
    if not match_found:
        matched.append(0)

# Print number of matches found
print("Number of matches:", sum(matched))

# Add matching result to dataframe
result = pd.concat([pred_data_title, test_data_title], axis=1)
result['title_match'] = matched

# Save result to CSV
result.to_csv("title_output-fuzzy.csv", index=False)

Number of matches: 487


#### Considering GT discrepency (wrong annotation (i.e., 5)), the total should be 492. So, F1 score: 98.4% (NeuralParsCit -- 97.45% & TransParsCit -- 90.7%). In the CORA-ref, the title field annotated as booktitle.

## Date Field Evaluation

In [ ]:
pred_date = []
for string in eval_data['date']:
  strings = string.strip()
  strings = strings.strip("['").strip("']").strip("'")
  strings = strings.lower()
  pred_date.append(strings)

pred_date = pd.DataFrame(pred_date, columns = ['pred-date'])

In [ ]:
test_data['date'].fillna('None', inplace=True)

test_date = []
characters_to_remove = ['.', ',', '(', ')', '[', ']', ':']
for string in test_data['date']:
  cleaned_string = string.strip().strip(''.join(characters_to_remove))
  normalize = cleaned_string.lower()
  test_date.append(normalize)

test_date = pd.DataFrame(test_date, columns = ['test-date'])

In [ ]:
matched_date = pd.DataFrame(columns=['date_match'])
matched_date['date_match'] = pred_date['pred-date'].eq(test_date['test-date']).replace([True, False], [1,0])
result = pd.concat([pred_date, test_date, matched_date], axis = 1, sort = False)
result.to_csv("date_output.csv", index=False)

In [ ]:
print(matched_date.value_counts())

date_match
1             485
0              15
Name: count, dtype: int64


### With fuzzy match, the total should be 494. F1 score: 98.80% (NeuralParsCit -- 98.93 & TransParsCit -- 87.7%)

## Author Field Evaluation

In [ ]:
pred_author = []
characters_to_remove = ['.']
for string in eval_data['author']:
  strings = string.strip()
  strings = strings.strip("['").strip("']")
  strings = strings.replace("'", "")
  cleaned_string = strings.rstrip(''.join(characters_to_remove))
  normalized = cleaned_string.lower()
  pred_author.append(normalized)

pred_author_data = pd.DataFrame(pred_author, columns = ['pred-author'])

In [ ]:
pred_author_data

In [ ]:
for i in pred_author_data['pred-author']:
  print(type(i))

In [ ]:
test_data['author'].fillna('None', inplace=True)

test_author = []
characters_to_remove = ['.']
for string in test_data['author']:
  cleaned_string = string.strip().rstrip(''.join(characters_to_remove))
  normalize = cleaned_string.lower()
  test_author.append(normalize)

test_author_data = pd.DataFrame(test_author, columns = ['test-author'])

In [ ]:
for i in test_author_data['test-author']:
  print(type(i))

In [ ]:
matched_author = pd.DataFrame(columns=['author_match'])
matched_author['author_match'] = pred_author_data['pred-author'].eq(test_author_data['test-author']).replace([True, False], [1,0])
result = pd.concat([pred_author_data, test_author_data, matched_author], axis = 1, sort = False)
result.to_csv("author_output.csv", index=False)

In [ ]:
print(matched_author.value_counts())

author_match
0               369
1               131
Name: count, dtype: int64


## Fuzzy Matching For Author Field

In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd


# Function to perform fuzzy string matching
def fuzzy_match(str1, str2):
    return fuzz.token_sort_ratio(str1.lower(), str2.lower())

# Perform string matching and fuzzy matching
matched = []
for pred_author in pred_author_data['pred-author']:
    match_found = False
    for test_author in test_author_data['test-author']:
        if pred_author.lower() == test_author.lower():  # Exact match
            matched.append(1)
            match_found = True
            break
        elif fuzzy_match(pred_author, test_author) >= 82:  # Fuzzy match with threshold 85%
            matched.append(1)
            match_found = True
            break
    if not match_found:
        matched.append(0)

# Print number of matches found
print("Number of matches:", sum(matched))

# Add matching result to dataframe
result = pd.concat([pred_author_data, test_author_data], axis=1)
result['author_match'] = matched

# Save result to CSV
result.to_csv("author_output-fuzzy.csv", index=False)

Number of matches: 488


#### Considering GT discrepency (author name annotated as editors). So, considering this wrong annotation (i.e, 9)), the total should be 497. So, F1 score: 99.4% (NeuralParsCit -- 99.1% & TransParsCit -- 98.8%)

##Venue Field Evaluation

In [ ]:
pred_venue = []
for string in eval_data['venue']:
  strings = string.strip()
  strings = strings.strip("['").strip("']").strip(".")
  strings = strings.lower()
  pred_venue.append(strings)

pred_data_venue = pd.DataFrame(pred_venue, columns = ['pred-venue'])

In [ ]:
pred_data_venue

,pred-venue
0,proc. 5th. bcs-facs refinement workshop
1,new generation computing
2,"technical report tu wien rr-02-92, version 2.0..."
3,proceedings of the 24th annual acm symposium o...
4,center for computer aids for industrial produc...
...,...
495,"logic colloquium'87, granada"
496,journal of artificial intelligence research
497,proceedings of the australian workshop on mach...
498,proc. post-conf. ws. on proof-theoretical exte...


In [ ]:
venue_data = pd.read_csv("/content/citation_venue_mapped.csv")

In [ ]:
venue_data['venue'].fillna('None', inplace=True)

test_venue = []
characters_to_remove = [',', '.']
for string in venue_data['venue']:
  cleaned_string = string.strip().rstrip(''.join(characters_to_remove))
  normalize_venue = cleaned_string.lower()
  test_venue.append(normalize_venue)

test_data_venue = pd.DataFrame(test_venue, columns = ['test-venue'])

In [ ]:
test_data_venue

,test-venue
0,proc. 5th. bcs-facs refinement workshop
1,new generation computing
2,"dept. for real-time systems, technical univers..."
3,in proceedings of the 24th annual acm symposiu...
4,center for computer aids for industrial produc...
...,...
495,"logic colloquium'87, granada"
496,journal of artificial intelligence research
497,in proceedings of the australian workshop on m...
498,proc. post-conf. ws. on proof-theoretical exte...


In [ ]:
matched_venue = pd.DataFrame(columns=['venue_match'])
matched_venue['venue_match'] = pred_data_venue['pred-venue'].eq(test_data_venue['test-venue']).replace([True, False], [1,0])
result = pd.concat([pred_data_venue, test_data_venue, matched_venue], axis = 1, sort = False)
result.to_csv("venue_output.csv", index=False)

In [ ]:
print(matched_venue.value_counts())

venue_match
1              285
0              215
Name: count, dtype: int64


##Fuzzy Matching For Venue Field

In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd


# Function to perform fuzzy string matching
def fuzzy_match(str1, str2):
    return fuzz.token_sort_ratio(str1.lower(), str2.lower())

# Perform string matching and fuzzy matching
matched = []
for pred_venue in pred_data_venue['pred-venue']:
    match_found = False
    for test_venue in test_data_venue['test-venue']:
        if pred_venue.lower() == test_venue.lower():  # Exact match
            matched.append(1)
            match_found = True
            break
        elif fuzzy_match(pred_venue, test_venue) >= 70:  # Fuzzy match with threshold 85%
            matched.append(1)
            match_found = True
            break
    if not match_found:
        matched.append(0)

# Print number of matches found
print("Number of matches:", sum(matched))

# Add matching result to dataframe
result = pd.concat([pred_data_venue, test_data_venue], axis=1)
result['venue_match'] = matched

# Save result to CSV
result.to_csv("venue_output-fuzzy.csv", index=False)

Number of matches: 463


#### We observed more strings did not match, further requires string normalization, such as, removing trailing or leading space, period, semicolon, colon, etc. Also, observed a lot of TN. For example, in the cases of pre-print, or thesis. If the evaluation is done again with solving the issues described, the result should be 484 correctly predicted. So, We will be looking at micro F1 score: 96.8% (NeuralParsCit -- micro F1 91.91%)